<a href="https://colab.research.google.com/github/fayexu/IDL_PJ_21Spring/blob/master/DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForMaskedLM
import torch

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-multilingual-cased") 
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-multilingual-cased")



In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    
)

In [ ]:
def predict_dist(line):
  res = []
  prediction = fill_mask(line)
  for p in prediction:
    res.append((p['token_str'].lower(), p['score']*100))
  return res

In [ ]:
t = predict_dist("The belly dancer dances on stage shaking w [MASK] hips and body.")
print(t)

[('both', 9.440796077251434), ('their', 3.255169466137886), ('the', 3.0517375096678734), ('慟', 2.414005622267723), ('big', 2.3398002609610558)]


In [ ]:
scores = []

In [ ]:
def get_bias_ratio(top_k):
  male_token = ["he", "his", "him", "himself", "cowboy", "cowboys", "male", "males", "father", "grandfather", "dad", "daddy" ,"son", "grandson", "man", "men", "waiter", "waiters", "gentleman", "gentlemen", "boy", "boys", "mr", "actor", "actors", "husband", "boyfriend", "brother", "brothers", "uncle", "uncles", "guy", "guys"]
  female_token = ["she", "her", "hers", "herself", "witch", "witches", "female", "females", "mother", "grandmother", "mom", "mum", "mommy", "daughter", "granddaughter", "woman", "women", "waitress", "waitresses", "lady", "ladies", "girl", "girls", "miss", "mrs", "actress", "actresses", "wife", "girlfriend", "sister", "sisters", "aunt", "aunts", "beauty"] 
  max_male_token = 'None'
  max_female_token = 'None'
  max_male_score = 0
  max_female_score = 0
  for token, score in top_k:
    if token in male_token:
      max_male_token = token
      max_male_score = score
      break
  for token, score in top_k:
    if token in female_token:
      max_female_token = token
      max_female_score = score
      break
  ratio = 0
  #if max_male_score > max_female_score:
  if max_male_token == 'None' and max_female_token == 'None':
    #print("no solution\n")
    return [0, 0];
  if max_male_score > max_female_score and max_female_score > 1:
    ratio = round(max_male_score / (max_female_score + max_male_score), 3)
    #print('Male dominates: bias score = {} with {} = {} and {} = {}\n'.format(ratio, max_male_token, max_male_score, max_female_token, max_female_score))
    print(ratio)
    scores.append(ratio)
    if ratio > 0.8:
      print()
    return [ratio, 0]
  elif max_male_score < max_female_score and max_male_score > 1:
    ratio = round(max_female_score / (max_female_score + max_male_score), 3)
    #print('Female dominates: bias score = {} with {} = {} and {} = {}\n'.format(1-ratio, max_female_token, max_female_score, max_male_token, max_male_score))
    print(1-ratio)
    scores.append(1-ratio)
    return [ratio, 1]
  return [0,0]

In [ ]:
print(t)
print(get_bias_ratio(t))

[('your', 18.149539828300476), ('his', 13.110914826393127), ('their', 7.546231895685196), ('the', 4.375532642006874), ('our', 3.6960341036319733)]
[0, 0]


In [ ]:
def getAverageRatio(input):
  #out = open(/content/"distribution.txt", "w")
  with open(input, 'r') as f:
    lines = f.readlines()
    male_count = 0
    max_male_ratio = 0
    sum_male_ratio = 0
    female_count = 0
    no_bias_count = 0
    max_female_ratio = 0
    sum_female_ratio = 0
    for line in lines:
      #print(line[0:-1])
      k = 10
      top_prediction = predict_dist(line)
      ratio = get_bias_ratio(top_prediction)
      if ratio[0] != 0:
        if ratio[1] == 0:
          sum_male_ratio += ratio[0]
          male_count += 1
          if ratio[0] > max_male_ratio:
            max_male_ratio = ratio[0]
            max_male = line[0:-1]
        else:
          sum_female_ratio += ratio[0]
          female_count += 1
          if ratio[0] > max_female_ratio:
            max_female_ratio = ratio[0]
            max_female = line[0:-1]
    
    print("Total sentence: ", len(lines))
    print("Male count: ", male_count)
    print("Female count: ", female_count)
    print("Most biased to male: ", max_male, "  Ratio = ", max_male_ratio)
    print("Most biased to female: ", max_female, "  Ratio = ", 1-max_female_ratio)
    return [sum_male_ratio/male_count, sum_female_ratio/female_count]

In [ ]:
male_dom, female_dom = getAverageRatio("/train_full_filtered.txt")
print("When male dominates: ", male_dom)
print("When female dominates: ", 1-female_dom)

0.819
0.646
0.854
0.742
0.838
0.713
0.832
0.947
0.765
0.717
0.652
0.669
0.731
0.819
0.833
0.606
0.29300000000000004
0.865
0.746
0.858
0.402
0.841
0.841
0.376
0.641
0.578
0.899
0.933
0.779
0.898
0.637
0.384
0.749
0.749
0.92
0.896
0.637
0.797
0.811
0.866
0.943
0.511
0.602
0.43100000000000005
0.695
0.907
0.824
0.531
0.959
0.854
0.804
0.742
0.797
0.737
0.869
0.904
0.488
0.692
0.488
0.953
0.953
0.32899999999999996
0.874
0.914
0.724
0.596
0.721
0.487
0.72
0.708
0.896
0.608
0.52
0.749
0.952
0.905
0.701
0.751
0.771
0.86
0.517
0.79
0.387
0.908
0.74
0.957
0.538
0.935
0.538
0.749
0.696
0.915
0.832
0.733
0.34299999999999997
0.849
0.821
0.711
0.604
0.765
0.547
0.912
0.894
0.85
0.839
0.482
0.899
0.813
0.537
0.759
0.7
0.81
0.31999999999999995
0.86
0.889
0.694
0.922
0.675
0.908
0.654
0.832
0.715
0.848
0.951
0.861
0.709
0.874
0.523
0.857
0.759
0.517
0.759
0.712
0.91
0.69
0.518
0.877
0.575
0.518
0.743
0.602
0.86
0.749
0.85
0.817
0.767
0.705
0.765
0.578
0.532
0.842
Total sentence:  663
Male count:  138
F